In [1]:
# --- 1. IMPORT THƯ VIỆN VÀ ĐỊNH NGHĨA HÀM ---

import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime, timezone
import collections
import joblib
import os

# QUAN TRỌNG: Định nghĩa lại hàm cyclical_encoder để joblib có thể tải pipeline
# (Copy từ notebook training)
def cyclical_encoder(X):
    X_encoded = np.array([])
    max_vals = [23, 6, 12]
    for i in range(X.shape[1]):
        col_data = X[:, i:i+1]
        max_val = max_vals[i]
        X_sin = np.sin(2 * np.pi * col_data / (max_val + 1))
        X_cos = np.cos(2 * np.pi * col_data / (max_val + 1))
        if X_encoded.size == 0:
            X_encoded = np.concatenate([X_sin, X_cos], axis=1)
        else:
            X_encoded = np.concatenate([X_encoded, X_sin, X_cos], axis=1)
    return X_encoded

print("✅ Các thư viện và hàm tùy chỉnh đã sẵn sàng.")

✅ Các thư viện và hàm tùy chỉnh đã sẵn sàng.


In [2]:
# --- 2. CÁC HÀM THU THẬP DỮ LIỆU ---

NODE_URL = "https://fullnode.mainnet.aptoslabs.com/v1"

def get_all_transactions(session, address):
    all_transactions = []
    start = 0
    limit = 100
    while True:
        params = {'start': start, 'limit': limit}
        try:
            response = session.get(f"{NODE_URL}/accounts/{address}/transactions", params=params)
            response.raise_for_status()
            transactions = response.json()
            if not transactions: break
            all_transactions.extend(transactions)
            start += len(transactions)
            if len(transactions) < limit: break
            time.sleep(0.1)
        except requests.exceptions.RequestException:
            break
    return all_transactions

def get_wallet_resources(session, address):
    try:
        response = session.get(f"{NODE_URL}/accounts/{address}/resources")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException:
        return []

# NÂNG CẤP: Hàm này được sửa lại để tạo ra các feature giống hệt lúc train
def create_feature_dataframe(session, address):
    all_transactions = get_all_transactions(session, address)
    resources = get_wallet_resources(session, address)

    # Khởi tạo profile với các giá trị mặc định
    profile = {
        'wallet_age_days': 0, 'apt_balance': 0, 'other_token_count': 0,
        'total_transaction_count': 0, 'successful_transaction_count': 0, 'failed_transaction_count': 0,
        'unique_interacted_contracts': 0, 'unique_interacted_addresses': 0,
        'avg_time_between_tx_seconds': -1, 'std_dev_time_between_tx_seconds': -1,
        'most_active_hour': -1, 'is_self_funded': 1,
        # Thêm các feature mới giống lúc train
        'tx_day_of_week': -1, 'tx_month': -1, 'tx_day_of_month': -1,
        'success_rate': 0, 'new_contract_rate': 0, 'balance_per_tx': 0
    }

    if not all_transactions:
        print(f"!!! CẢNH BÁO: Không tìm thấy giao dịch nào cho ví {address}")
        return pd.DataFrame([profile])

    # Tính toán các feature cơ bản
    profile['total_transaction_count'] = len(all_transactions)
    profile['successful_transaction_count'] = sum(1 for tx in all_transactions if tx['success'])
    profile['failed_transaction_count'] = profile['total_transaction_count'] - profile['successful_transaction_count']

    # Tính toán các feature thời gian
    first_tx = all_transactions[-1]
    first_tx_timestamp = int(first_tx['timestamp']) // 1000000
    creation_datetime = datetime.fromtimestamp(first_tx_timestamp, tz=timezone.utc)
    profile['wallet_age_days'] = (datetime.now(timezone.utc) - creation_datetime).days
    profile['tx_day_of_week'] = creation_datetime.dayofweek # 0=Thứ 2, 6=Chủ Nhật
    profile['tx_month'] = creation_datetime.month
    profile['tx_day_of_month'] = creation_datetime.day

    timestamps = sorted([int(tx['timestamp']) // 1000000 for tx in all_transactions])
    hours = [datetime.fromtimestamp(ts, tz=timezone.utc).hour for ts in timestamps]
    if hours:
        profile['most_active_hour'] = collections.Counter(hours).most_common(1)[0][0]

    # Các feature khác
    if first_tx.get('sender') != address:
        profile['is_self_funded'] = 0

    apt_balance = 0
    other_token_count = 0
    for resource in resources:
        if resource['type'] == '0x1::coin::CoinStore<0x1::aptos_coin::AptosCoin>':
            apt_balance = int(resource['data']['coin']['value']) / 10**8
        elif resource['type'].startswith('0x1::coin::CoinStore<'):
            other_token_count += 1
    profile['apt_balance'] = apt_balance
    profile['other_token_count'] = other_token_count

    interacted_contracts = {tx['payload']['function'].split('::')[0] for tx in all_transactions if tx['payload'].get('function')}
    profile['unique_interacted_contracts'] = len(interacted_contracts)

    # Tính các feature tỷ lệ
    profile['success_rate'] = profile['successful_transaction_count'] / (profile['total_transaction_count'] + 1e-6)
    profile['new_contract_rate'] = profile['unique_interacted_contracts'] / (profile['total_transaction_count'] + 1e-6)
    profile['balance_per_tx'] = profile['apt_balance'] / (profile['total_transaction_count'] + 1e-6)

    return pd.DataFrame([profile])

print("✅ Các hàm thu thập dữ liệu đã sẵn sàng.")

✅ Các hàm thu thập dữ liệu đã sẵn sàng.


In [3]:
# --- 3. TẢI PIPELINE AI ---

PIPELINE_PATH = 'C:/A-A-C/models/aptos_pro_pipeline.joblib'

try:
    # Chỉ cần tải pipeline, không cần scaler riêng nữa
    loaded_pipeline = joblib.load(PIPELINE_PATH)
    print("✅ Tải thành công pipeline AI.")
except FileNotFoundError:
    print(f"❌ LỖI: Không tìm thấy file pipeline tại '{PIPELINE_PATH}'.")
    print("   Vui lòng chạy notebook 'train_aptos.ipynb' để tạo file pipeline trước.")
    loaded_pipeline = None

✅ Tải thành công pipeline AI.


In [4]:
# --- 4. NHẬP VÍ VÀ DỰ ĐOÁN ---

if loaded_pipeline:
    # ĐIỀN ĐỊA CHỈ VÍ BẠN MUỐN KIỂM TRA VÀO ĐÂY
    wallet_to_predict = "0x3a1f89c3768e0ec0fa92aeafbc5bb233e1ae0863dc33cd79f1f0f1e1acdad42b"

    print(f"--- Bắt đầu lấy dữ liệu và tạo feature cho ví: {wallet_to_predict} ---")
    session = requests.Session()
    # 1. Tạo DataFrame feature thô
    wallet_df = create_feature_dataframe(session, wallet_to_predict)

    print("\n--- Thực hiện dự đoán bằng pipeline ---")
    # 2. Đưa thẳng DataFrame vào pipeline. Pipeline sẽ tự động xử lý mọi thứ.
    prediction = loaded_pipeline.predict(wallet_df)
    prediction_proba = loaded_pipeline.predict_proba(wallet_df)

    # --- Hiển thị kết quả ---
    print("\n================ KẾT QUẢ DỰ ĐOÁN ================")
    result_label = "SYBIL (Đáng nghi)" if prediction[0] == 1 else "NORMAL (Người dùng thường)"
    confidence = prediction_proba[0][prediction[0]] * 100

    print(f"Ví: {wallet_to_predict}")
    print(f"==> Kết quả: {result_label}")
    print(f"==> Độ tin cậy của mô hình: {confidence:.2f}%")
    print("-------------------------------------------------")
    print(f"Xác suất là Normal (0): {prediction_proba[0][0]*100:.2f}%")
    print(f"Xác suất là Sybil (1):  {prediction_proba[0][1]*100:.2f}%")
    print("=================================================")
else:
    print("❌ Không thể dự đoán vì pipeline chưa được tải.")

--- Bắt đầu lấy dữ liệu và tạo feature cho ví: 0x3a1f89c3768e0ec0fa92aeafbc5bb233e1ae0863dc33cd79f1f0f1e1acdad42b ---
!!! CẢNH BÁO: Không tìm thấy giao dịch nào cho ví 0x3a1f89c3768e0ec0fa92aeafbc5bb233e1ae0863dc33cd79f1f0f1e1acdad42b

--- Thực hiện dự đoán bằng pipeline ---

================ KẾT QUẢ DỰ ĐOÁN ================
Ví: 0x3a1f89c3768e0ec0fa92aeafbc5bb233e1ae0863dc33cd79f1f0f1e1acdad42b
==> Kết quả: SYBIL (Đáng nghi)
==> Độ tin cậy của mô hình: 65.10%
-------------------------------------------------
Xác suất là Normal (0): 34.90%
Xác suất là Sybil (1):  65.10%
